# Local Pipeline run

In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import numpy as np

In [2]:
import sys
print(sys.path)

['/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages/IPython/extensions', '/home/michal/.ipython']


### Setup

#### Update environment PATH to include TFX CLI and skaffold

In [3]:
import yaml

# Set `PATH` to include the directory containing TFX CLI and skaffold.
PATH=%env PATH
HOME=%env HOME
#PYTHONPATH=%env PYTHONPATH 

%env PATH={HOME}/.local/bin:{PATH}
#%env PYTHONPATH=/home/michal/PycharmProjects/ml-gcp-pipeline:{PYTHONPATH}

env: PATH=/home/michal/.local/bin:/home/michal/venv/ML-3.8/bin:/home/michal/google-cloud-sdk/bin:/home/michal/anaconda3/bin:/home/michal/anaconda3/condabin:/home/michal/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


#### Validate lab package version installation

In [4]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.25.0
KFP version: 1.0.4


**Note**: this lab was built and tested with the following package versions:

`TFX version: 0.25.0`  
`KFP version: 1.0.4`

## Setup local path to data, train, test folders 

In [5]:
import os
from pathlib import Path

try: notebook_path
except NameError: 
    notebook_path=os.getcwd() # to only run it once

local_data_dirpath = os.path.join(notebook_path, 'data')

local_train_dirpath = os.path.join(local_data_dirpath, "train")
local_train_filepath = os.path.join(local_train_dirpath, "train.csv")
local_test_dirpath = os.path.join(local_data_dirpath, "test")
local_test_filepath = os.path.join(local_test_dirpath, "test.csv")


## Load kaggle

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!rm kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


## Download data from kaggle, unzip it and copy it to data folder

In [ ]:
!kaggle competitions download -c titanic -p {local_data_dirpath} --force
!unzip -o {local_data_dirpath}/"titanic.zip" -d {local_data_dirpath}
!cp {local_data_dirpath}/"train.csv" {local_train_filepath}
!cp {local_data_dirpath}/"test.csv" {local_test_filepath}

# clean up
!rm  {local_data_dirpath}/*.csv  {local_data_dirpath}/*.zip

## Copy data to gs

In [6]:
PIPELINE_NAME = 'tfx-titanic-training'
MODEL_NAME = 'tfx_titanic_classifier'
DATA_ROOT_URI = local_train_dirpath
RUNTIME_VERSION = '2.3'
PYTHON_VERSION = '3.7'
ENABLE_CACHE=True

TRAIN_STEPS = '10000'
EVAL_STEPS = '1000'
TRAIN_BATCH_SIZE = '64'
EVAL_BATCH_SIZE = '64'

ENABLE_TUNING=True
TUNER_STEPS = '2000'
MAX_TRIALS = '10'


In [7]:
%env PIPELINE_NAME={PIPELINE_NAME}
%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}
%env ENABLE_CACHE={ENABLE_CACHE}

%env TRAIN_STEPS={TRAIN_STEPS}
%env EVAL_STEPS={EVAL_STEPS}
%env TRAIN_BATCH_SIZE={TRAIN_BATCH_SIZE}
%env EVAL_BATCH_SIZE={EVAL_BATCH_SIZE}


%env ENABLE_TUNING={ENABLE_TUNING}
%env TUNER_STEPS={TUNER_STEPS}
%env MAX_TRIALS={MAX_TRIALS}


env: PIPELINE_NAME=tfx-titanic-training
env: MODEL_NAME=tfx_titanic_classifier
env: DATA_ROOT_URI=/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/data/train
env: RUNTIME_VERSION=2.3
env: PYTHON_VERSION=3.7
env: ENABLE_CACHE=True
env: TRAIN_STEPS=10000
env: EVAL_STEPS=1000
env: TRAIN_BATCH_SIZE=64
env: EVAL_BATCH_SIZE=64
env: ENABLE_TUNING=True
env: TUNER_STEPS=2000
env: MAX_TRIALS=10


## Local pipeline run

In [6]:
%cd {notebook_path}/pipeline

/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/pipeline


In [9]:
!python local_runner.py

2021-04-25 22:59:20.937904: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:absl:PIPELINE_ROOT=/home/michal/artifact-store/tfx-titanic-training/20210425_225924
INFO:absl:Cleaning local log folder : /tmp/logs
INFO:absl:train_steps for training: 10000
INFO:absl:tuner_steps for tuning: 2000
INFO:absl:data_root_uri for training: /home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/data/train
INFO:absl:eval_steps for evaluating: 1000
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:tuner_args: {'module_file': 'model.py', 'examples': Channel(
    type_name: Examples
    artifacts: []
), 'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: []
), 'train_args': {'num_steps': 2000}, 'eval_args': {'num_steps': 1000}, 'custom_config': {'m

INFO:absl:TFT preprocessing. Mean value for Age = Tensor("mean/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Fare = Tensor("mean_1/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Age = Tensor("mean/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Fare = Tensor("mean_1/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:Feature Embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Ticket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Parch has no shape. Setting

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:absl:Feature Embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Ticket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Name has no shape. Setting to VarLenSparseTensor.

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Transform is finished.
INFO:absl:Component ExampleValidator is running.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Validation complete for split train. Anomalies written to /home/michal/artifact-store/tfx-titanic-training/20210425_225924/ExampleValidator/anomalies/6/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/michal/artifact-store/tfx-titanic-training/20210425_225924/ExampleValidator/anomalies/6/eval.
INF

INFO:absl:Feature Age_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Embarked_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Fare_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Parch_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Pclass_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Sex_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature SibSp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Survived_xf has a shape . Setting to DenseTensor.
Search space summary
Default search space size: 5
learning_rate (Float)
{'default': 0.0009167702421017742, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
n_layers (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
n_units_1 (Int)
{'default': 72, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'n_layers', 'values': [1]}}], 'min_value': 8, 'max_value': 128, 'step': 8, 'samp

...

2000/2000 [==============================] - 11s 5ms/step - loss: 0.5313 - tp: 21071.0000 - fp: 4718.0000 - tn: 76062.0000 - fn: 26149.0000 - binary_accuracy: 0.7589 - precision: 0.8171 - recall: 0.4462 - auc: 0.8106 - val_loss: 0.4453 - val_tp: 19575.0000 - val_fp: 4894.0000 - val_tn: 30494.0000 - val_fn: 9037.0000 - val_binary_accuracy: 0.7823 - val_precision: 0.8000 - val_recall: 0.6842 - val_auc: 0.8969

Trial 6 Complete [00h 00m 12s]
val_binary_accuracy: 0.7823281288146973

Best val_binary_accuracy So Far: 0.794140636920929
Total elapsed time: 00h 01m 14s

Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.00016704        |0.00082761        
n_layers          |2                 |1                 
n_units_1         |104               |80                
n_units_2         |72                |None              

INFO:absl:Model: "functional_1"
INFO:absl:______________________________________________________________________________

2000/2000 [==============================] - 11s 6ms/step - loss: 0.4843 - tp: 27619.0000 - fp: 6713.0000 - tn: 74065.0000 - fn: 19603.0000 - binary_accuracy: 0.7944 - precision: 0.8045 - recall: 0.5849 - auc: 0.8283 - val_loss: 0.4090 - val_tp: 21458.0000 - val_fp: 5645.0000 - val_tn: 29744.0000 - val_fn: 7153.0000 - val_binary_accuracy: 0.8000 - val_precision: 0.7917 - val_recall: 0.7500 - val_auc: 0.8931

Trial 8 Complete [00h 00m 12s]
val_binary_accuracy: 0.8000312447547913

Best val_binary_accuracy So Far: 0.8000312447547913
Total elapsed time: 00h 01m 40s

Search: Running Trial #9

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.00050472        |0.0014422         
n_layers          |2                 |1                 
n_units_1         |72                |32                
n_units_2         |120               |None              

INFO:absl:Model: "functional_1"
INFO:absl:_____________________________________________________________________________

2000/2000 [==============================] - 11s 5ms/step - loss: 0.5097 - tp: 24407.0000 - fp: 5336.0000 - tn: 75443.0000 - fn: 22814.0000 - binary_accuracy: 0.7801 - precision: 0.8206 - recall: 0.5169 - auc: 0.8195 - val_loss: 0.4227 - val_tp: 20700.0000 - val_fp: 5273.0000 - val_tn: 30121.0000 - val_fn: 7906.0000 - val_binary_accuracy: 0.7941 - val_precision: 0.7970 - val_recall: 0.7236 - val_auc: 0.8966

Trial 10 Complete [00h 00m 12s]
val_binary_accuracy: 0.7940781116485596

Best val_binary_accuracy So Far: 0.8000312447547913
Total elapsed time: 00h 02m 05s
INFO:absl:Finished tuning... Tuner ID: tuner0
Results summary
Results in /home/michal/artifact-store/tfx-titanic-training/20210425_225924/.temp/8/titanic_tuning
Showing 10 best trials
Objective(name='val_binary_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0014421563688159582
n_layers: 1
n_units_1: 32
Score: 0.8000312447547913
Trial summary
Hyperparameters:
learning_rate: 0.0008276127298347325
n_la

Epoch 1/10
2021-04-25 23:01:45.268828: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2021-04-25 23:01:45.277369: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /tmp/logs/train/plugins/profile/2021_04_25_23_01_45
2021-04-25 23:01:45.278937: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to /tmp/logs/train/plugins/profile/2021_04_25_23_01_45/michal-ubuntu20-vm.trace.json.gz
2021-04-25 23:01:45.281979: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /tmp/logs/train/plugins/profile/2021_04_25_23_01_45
2021-04-25 23:01:45.282080: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for memory_profile.json.gz to /tmp/logs/train/plugins/profile/2021_04_25_23_01_45/micha

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:Evaluation complete. Results written to /home/michal/artifact-store/tfx-titanic-training/20210425_225924/Evaluator/evaluation/10.
INFO:absl:Checking validation results.
INFO:absl:Blessing result False written to /home/michal/artifact-store/tfx-titanic-training/20210425_225924/Evaluator/blessing/10.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Evaluator is finished.
INFO:absl:Component InfraValidator is running.
INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
2021-04-25 23:02:58.992232: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:588] No property is defined for the Type
INFO:absl:Running executor for InfraValidator
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl

Run docker container for serving 

In [36]:
!docker run --rm -p 8500:8500 -p 8501:8501 -p 8503:8503 -v=1 \
 --mount type=bind,source=/home/michal/artifact-store/tfx-titanic-training/20210416_133756/Pusher/pushed_model/,target=/models/tfx_titanic_classifier \
 -e MODEL_NAME=tfx_titanic_classifier -t tensorflow/serving:latest
                

                

2021-04-10 20:30:48.816717: I tensorflow_serving/model_servers/server.cc:88] Building single TensorFlow model file config:  model_name: tfx_titanic_classifier model_base_path: /models/tfx_titanic_classifier
2021-04-10 20:30:48.816899: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2021-04-10 20:30:48.816950: I tensorflow_serving/model_servers/server_core.cc:587]  (Re-)adding model: tfx_titanic_classifier
2021-04-10 20:30:48.917580: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917636: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917653: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917703: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32]

Functions for serializing data to tf.train.Example

In [35]:
import importlib
import client.client_util as client_util


importlib.reload(client_util)


df = client_util.read_csv_for_prediction(local_test_filepath)
df.head()

df_first_10 = df.head(10)
df_first_10_dict = df_first_10.to_dict(orient='records')


df_dict = df.to_dict(orient='records')


df_first_10_dict

[{'PassengerId': 892,
  'Pclass': 3,
  'Name': 'Kelly, Mr. James',
  'Sex': 'male',
  'Age': 34.5,
  'SibSp': 0,
  'Parch': 0,
  'Ticket': '330911',
  'Fare': 7.8292,
  'Cabin': '',
  'Embarked': 'Q'},
 {'PassengerId': 893,
  'Pclass': 3,
  'Name': 'Wilkes, Mrs. James (Ellen Needs)',
  'Sex': 'female',
  'Age': 47.0,
  'SibSp': 1,
  'Parch': 0,
  'Ticket': '363272',
  'Fare': 7.0,
  'Cabin': '',
  'Embarked': 'S'},
 {'PassengerId': 894,
  'Pclass': 2,
  'Name': 'Myles, Mr. Thomas Francis',
  'Sex': 'male',
  'Age': 62.0,
  'SibSp': 0,
  'Parch': 0,
  'Ticket': '240276',
  'Fare': 9.6875,
  'Cabin': '',
  'Embarked': 'Q'},
 {'PassengerId': 895,
  'Pclass': 3,
  'Name': 'Wirz, Mr. Albert',
  'Sex': 'male',
  'Age': 27.0,
  'SibSp': 0,
  'Parch': 0,
  'Ticket': '315154',
  'Fare': 8.6625,
  'Cabin': '',
  'Embarked': 'S'},
 {'PassengerId': 896,
  'Pclass': 3,
  'Name': 'Hirvonen, Mrs. Alexander (Helga E Lindqvist)',
  'Sex': 'female',
  'Age': 22.0,
  'SibSp': 1,
  'Parch': 1,
  'Ticket':

Use Prod client

In [ ]:
import client.prod_client as prod_client
import client.client_util as client_util

import importlib
importlib.reload(prod_client)
importlib.reload(client_util)

prodclient = prod_client.ProdClient(host='localhost:8500', model_name='tfx_titanic_classifier', \
                                    model_version=12, \
                                    in_tensor_name='examples', signature_name='serving_default')


#prodclient.predict(request_data=df_first_10_dict, request_timeout=30)

all_predition = prodclient.predict(request_data=df_dict, request_timeout=30)
all_predition


Use InMemory Client

In [16]:
import client.inmemory_client as inmemory_client
import client.client_util as client_util

import importlib
importlib.reload(inmemory_client)
importlib.reload(client_util)

inmemory_client = inmemory_client.InMemoryClient(
            'tests/resources/serving_model/1618577937', in_tensor_name='examples', signature_name='serving_default')



#prediction = inmemory_client.predict(request_data=df_first_10_dict)
#prediction
all_predition = inmemory_client.predict(request_data=df_dict, request_timeout=30)
#all_predition

all_predition_numpy = all_predition['output_0'].numpy()


In [27]:
import numpy as np
all_predition_numpy_int = np.rint(all_predition_numpy).astype(int)
#all_predition_numpy.astype(int)

### Construct submission answer

In [32]:
df_submisssion = df.copy()
df_submisssion['Survived_probs'] = all_predition_numpy
df_submisssion['Survived'] = all_predition_numpy_int
#df_submisssion

df_test_prediction_to_submit = df_submisssion[['PassengerId', 'Survived']]
df_test_prediction_to_submit.head()


df_test_prediction_to_submit.to_csv('titanic_tf_submission_1.csv',index=False)

In [34]:
df_submisssion.head(30)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_probs,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q,0.136743,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,,S,0.440951,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,,Q,0.130714,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,,S,0.099841,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,,S,0.670943,1
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,,S,0.147914,0
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,,Q,0.711517,1
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,,S,0.249630,0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,,C,0.758666,1
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,,S,0.039475,0
